In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb

In [2]:
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix, cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import matthews_corrcoef
from sklearn.model_selection import train_test_split

In [3]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import ExtraTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score, cross_val_predict

C:\Users\User\anaconda3\Lib\site-packages\dask\dataframe\_pyarrow_compat.py:23: UserWarning: You are using pyarrow version 11.0.0 which is known to be insecure. See https://www.cve.org/CVERecord?id=CVE-2023-47248 for further details. Please upgrade to pyarrow>=14.0.1 or install pyarrow-hotfix to patch your current version.
  warnings.warn(


In [4]:
df = pd.read_csv('protine_apaac.csv')
df.head()

,0,1.938389326,0.1,1.938389326.1,0.969194663,0.2,0.3,1.938389326.2,1.938389326.3,0.4,...,0.969194663.3,1.938389326.5,1.938389326.6,0.969194663.4,0.969194663.5,0.969194663.6,0.015108932,0.011875905,0.005111392,-0.001290892
0,0,0.000000,0.000000,1.032164,0.000000,0.000000,0.0,3.096492,1.032164,0.000000,...,2.064328,0.000000,2.064328,0.000000,2.064328,0.000000,0.010461,-0.000431,-0.017648,-0.024546
1,0,0.932778,0.932778,0.000000,0.932778,0.000000,0.0,1.865556,1.865556,0.000000,...,0.000000,0.932778,0.932778,0.000000,0.932778,0.000000,0.021136,0.024491,0.002803,0.018792
2,0,3.082700,0.000000,0.000000,0.000000,1.027567,0.0,1.027567,1.027567,1.027567,...,1.027567,1.027567,1.027567,1.027567,0.000000,2.055134,-0.007817,-0.015103,-0.004088,-0.000559
3,0,3.067533,1.022511,0.000000,1.022511,0.000000,0.0,3.067533,0.000000,1.022511,...,2.045022,0.000000,1.022511,0.000000,0.000000,3.067533,-0.033603,-0.021546,0.017276,0.015362
4,0,0.000000,0.000000,0.000000,1.020452,1.020452,0.0,2.040904,3.061357,0.000000,...,2.040904,2.040904,1.020452,0.000000,0.000000,4.081809,0.002407,0.007620,-0.015185,-0.015295


In [5]:
y = df['0']
x = df.drop('0', axis=1) 

In [6]:
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.3)

In [7]:
total_Metics = []
total_Metics = pd.DataFrame(total_Metics)
total_Metics['Classifier'] = 'Classifier'
total_Metics['Accuracy'] = 'Accuracy'
total_Metics['mcc'] = 'mcc'
# total_Metics['auc'] = 'auc'
total_Metics['Kappa'] = 'Kappa'
total_Metics['precision'] = 'precision'
total_Metics['recall'] = 'recall'
total_Metics['f1'] = 'f1'
total_Metics['sensitivity'] = 'sensitivity'
total_Metics['specificity'] = 'specificity'

models = [RandomForestClassifier(n_estimators = 200, max_depth = 10),
          XGBClassifier(n_estimators = 200,max_depth = 10, learning_rate = 0.1),
          LGBMClassifier(learning_rate = 0.1,max_depth = 10,random_state = 50),
          GradientBoostingClassifier(n_estimators = 200, learning_rate = 0.5, random_state = 50),
          AdaBoostClassifier(n_estimators = 200, learning_rate = 0.1, random_state = 50)]
cv = KFold(n_splits=5, random_state=1, shuffle=True)
for model in models:
  from sklearn.metrics import f1_score, precision_score, recall_score, log_loss, accuracy_score, matthews_corrcoef, roc_auc_score, cohen_kappa_score
  # evaluate model
  # scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
  # model.fit(xtrain, ytrain)
  # pred = model.predict(xtest)
  pred = cross_val_predict(model, xtrain, ytrain, cv=cv, n_jobs=-1)

  # cm1 = confusion_matrix(y, y_pred)
  # report performance
  Accuracy = accuracy_score(ytrain, pred)
  mcc = matthews_corrcoef(ytrain, pred)
  cm1 = confusion_matrix(ytrain, pred)
  kappa = cohen_kappa_score(ytrain, pred)
  f1 = f1_score(ytrain, pred)
  precision_score = precision_score(ytrain, pred)
  recall_score = recall_score(ytrain, pred)
  sensitivity = cm1[0,0]/(cm1[0,0]+cm1[0,1])
  specificity = cm1[1,1]/(cm1[1,0]+cm1[1,1])
  # y_pred = np.argmax(y_pred, axis=0)
  # auc = roc_auc_score(y, y_pred, multi_class='ovr')
  total_Metics.loc[len(total_Metics.index)] = [model,Accuracy, mcc, kappa, precision_score,recall_score, f1, sensitivity,specificity]

print(total_Metics)

                                          Classifier  Accuracy       mcc  \
0  RandomForestClassifier(max_depth=10, n_estimat...  0.626881  0.032892   
1  XGBClassifier(base_score=None, booster=None, c...  0.584754  0.031051   
2      LGBMClassifier(max_depth=10, random_state=50)  0.606820  0.076490   
3  GradientBoostingClassifier(learning_rate=0.5, ...  0.583751  0.050381   
4  AdaBoostClassifier(learning_rate=0.1, n_estima...  0.630893  0.081490   

      Kappa  precision    recall        f1  sensitivity  specificity  
0  0.022461   0.412500  0.092179  0.150685     0.926448     0.092179  
1  0.029923   0.385246  0.262570  0.312292     0.765258     0.262570  
2  0.072901   0.426087  0.273743  0.333333     0.793427     0.273743  
3  0.049549   0.397849  0.310056  0.348509     0.737089     0.310056  
4  0.066171   0.460938  0.164804  0.242798     0.892019     0.164804  


Over Sampling: RandomOverSampler

In [9]:
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=42)
x_ros, y_ros = ros.fit_resample(xtrain, ytrain)

In [10]:
total_Metics = []
total_Metics = pd.DataFrame(total_Metics)
total_Metics['Classifier'] = 'Classifier'
total_Metics['Accuracy'] = 'Accuracy'
total_Metics['mcc'] = 'mcc'
# total_Metics['auc'] = 'auc'
total_Metics['Kappa'] = 'Kappa'
total_Metics['precision'] = 'precision'
total_Metics['recall'] = 'recall'
total_Metics['f1'] = 'f1'
total_Metics['sensitivity'] = 'sensitivity'
total_Metics['specificity'] = 'specificity'

models = [RandomForestClassifier(n_estimators = 200, max_depth = 10),
          XGBClassifier(n_estimators = 200,max_depth = 10, learning_rate = 0.1),
          LGBMClassifier(learning_rate = 0.1,max_depth = 10,random_state = 50),
          GradientBoostingClassifier(n_estimators = 200, learning_rate = 0.5, random_state = 50),
          AdaBoostClassifier(n_estimators = 200, learning_rate = 0.1, random_state = 50)]
cv = KFold(n_splits=5, random_state=1, shuffle=True)
for model in models:
  from sklearn.metrics import f1_score, precision_score, recall_score, log_loss, accuracy_score, matthews_corrcoef, roc_auc_score, cohen_kappa_score
  # evaluate model
  # scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
  # model.fit(xtrain, ytrain)
  # pred = model.predict(xtest)
  pred = cross_val_predict(model, x_ros, y_ros, cv=cv, n_jobs=-1)

  # cm1 = confusion_matrix(y, y_pred)
  # report performance
  Accuracy = accuracy_score(y_ros, pred)
  mcc = matthews_corrcoef(y_ros, pred)
  cm1 = confusion_matrix(y_ros, pred)
  kappa = cohen_kappa_score(y_ros, pred)
  f1 = f1_score(y_ros, pred)
  precision_score = precision_score(y_ros, pred)
  recall_score = recall_score(y_ros, pred)
  sensitivity = cm1[0,0]/(cm1[0,0]+cm1[0,1])
  specificity = cm1[1,1]/(cm1[1,0]+cm1[1,1])
  # y_pred = np.argmax(y_pred, axis=0)
  # auc = roc_auc_score(y, y_pred, multi_class='ovr')
  total_Metics.loc[len(total_Metics.index)] = [model,Accuracy, mcc, kappa, precision_score,recall_score, f1, sensitivity,specificity]

print(total_Metics)

                                          Classifier  Accuracy       mcc  \
0  RandomForestClassifier(max_depth=10, n_estimat...  0.740219  0.480523   
1  XGBClassifier(base_score=None, booster=None, c...  0.725352  0.451050   
2      LGBMClassifier(max_depth=10, random_state=50)  0.739437  0.479513   
3  GradientBoostingClassifier(learning_rate=0.5, ...  0.715962  0.433906   
4  AdaBoostClassifier(learning_rate=0.1, n_estima...  0.601721  0.203683   

      Kappa  precision    recall        f1  sensitivity  specificity  
0  0.480438   0.735791  0.749609  0.742636     0.730829     0.749609  
1  0.450704   0.716867  0.744914  0.730622     0.705790     0.744914  
2  0.478873   0.727679  0.765258  0.745995     0.713615     0.765258  
3  0.431925   0.697143  0.763693  0.728902     0.668232     0.763693  
4  0.203443   0.597015  0.625978  0.611154     0.577465     0.625978  
